# Examples of Combining Core Modules

By combining `LLMPromptProcessor` with RAG components, you can create RAG-based chatbots. The example below is not fully optimized but serves to illustrate how RAG chatbots work in practice.

In [ ]:
# Auto reload imports
%reload_ext autoreload
%autoreload 2

# Visualization
%matplotlib inline

# Imports
import pandas as pd
import os

In [ ]:
example_urls = [
    "https://www.lynxanalytics.com/generative-ai-platform",
    "https://en.wikipedia.org/wiki/GPT-3",
    "https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)",
    "https://en.wikipedia.org/wiki/Generative_pre-trained_transformer",
    "https://en.wikipedia.org/wiki/Large_language_model",
    "https://arxiv.org/pdf/1706.03762",
]

example_locations = [
    '../../data/example_doc_to_parse/ChatGPT - Wikipedia.html', 
    '../../README.md', 
    '../../data/example_doc_to_parse/wiki_GPT2.txt', 
    '../../data/example_doc_to_parse/2402.06196.pdf'
]

You can replace the `{context}` placeholder with the top retrieved content to create a simple RAG-style chatbot.

In [ ]:
from sziaaitls.core.document_library import DocumentLibrary
from sziaaitls.core.rag_solutions import SimpleRAG
from sziaaitls.core.vector_stores import VectorStore
from sziaaitls.core.embedders import TextEmbedder
from sziaaitls.core.llms import LLMPromptProcessor, Message, ChatCompletionPrompt

mydoclib = await DocumentLibrary.create(
    urls=example_urls, # can be empty
    file_paths=example_locations, # can be empty
    chunk_method="simple_intervals",
    max_chunk_size=768,
    min_chunk_size=256,
    min_leftover_chunk_size=128,
    ignore_single_newline=True,
)

# initializing a vector store and an embedder
vector_store = VectorStore(name="usearch", dimension=1536)
embedder = TextEmbedder("openai", api_key=os.environ.get("OPENAI_API_KEY"), model="text-embedding-3-small", rate_limit=3000, period=60, max_batch_tokens=32768)

# creating a SimpleRAG instance
simple_rag = await SimpleRAG.create(mydoclib, embedder, vector_store) # using the document library from above
print(f"RAG Store is created with {simple_rag.next_id -1} items.")


pd.DataFrame(simple_rag.metadata_list).head(2)

In [ ]:
prompt_list = [
    {'role': 'system', 'content': 'You are a helpful assistant that answers questions based on the provided context.'},
    {'role': 'system', 'content': 'Please phrase your answer solely based on the following context:\n\n{context}.'},
    {'role': 'user', 'content': '{question}'}
]

async def RAG_based_answer(
        question: str, 
        rag_kb: SimpleRAG, 
        chatbot_prompt: list[dict], 
        max_context_size_token: int=2000, 
        max_context_size_chunks: int=20) -> str:

    context_es = await rag_kb.search_query(question, max_results=max_context_size_chunks, token_limit=max_context_size_token)
    context_txt = "\n\n--------------------------------------\n\n".join([finding.embedding.embedding_content for finding in context_es])
    
    prompt = ChatCompletionPrompt([Message.from_dict(p).format(question=question, context=context_txt) for p in chatbot_prompt])

    llm = LLMPromptProcessor(
        api_key=os.environ.get("OPENAI_API_KEY"),
        model="gpt-4o-mini",
        rate_limit = 350,
        period = 60,
    )

    answer = await llm.acreate(prompt)

    return answer.content
    
# Example question
question = "What is the main idea behind GPT-3?"
answer = await RAG_based_answer(
    question=question, 
    rag_kb=simple_rag, 
    chatbot_prompt=prompt_list, 
    max_context_size_token=2000, 
    max_context_size_chunks=20
)

In [ ]:
print(f"Question: {question}\nAnswer: {answer}")

**F.I.N**